<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3>Part 1</h3>
<li>Start by creating a new Notebook for this assignment.</li>
<li>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe</li>
<li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
<li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
<li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>

In [1]:
import pandas as pd
import numpy as np 

scrapes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', index_col=0, attrs={"class":"wikitable"})

#There were multiple tables but only the first is needed

df_postal = pd.DataFrame(scrapes[0])
df_postal.head()

,Borough,Neighborhood
Postal code,,
M1A,Not assigned,NaN
M2A,Not assigned,NaN
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront


In [2]:
df_postal = df_postal.dropna()
df_postal

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...
M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business reply mail Processing CentrE


In [3]:
df_postal.shape

(103, 2)

<h3>Part 2</h3>
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [4]:
df_location = pd.read_csv('Geospatial_Coordinates.csv')
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df_canada = df_postal.join(df_location.set_index('Postal Code'))
df_canada.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [6]:
df_toronto = df_canada[df_canada.Borough.str.contains('Toronto')]
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
M4E,East Toronto,The Beaches,43.676357,-79.293031


In [7]:
df_toronto.shape

(39, 4)

<h3>Part 3</h3>
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
Just make sure:
<li>to add enough Markdown cells to explain what you decided to do and to report any observations you make.</li>
<li>to generate maps to visualize your neighborhoods and how they cluster together.</li>

In [8]:
import folium

map_Toronto = folium.Map(location=(df_toronto['Latitude'][0],df_toronto['Longitude'][0]), zoom_start=12.4)
for each in range(0, len(df_toronto.index)):
    label = '{}, {}'.format(df_toronto['Neighborhood'][each],df_toronto['Borough'][each])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [df_toronto['Latitude'][each],df_toronto['Longitude'][each]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

map_Toronto

In [9]:
# @hidden_cell
CLIENT_ID = 'QWVH1ATFD2XLOHHXNGXCXZUWFIHKK1E15R5UEITWQNCYTFPO'
CLIENT_SECRET = 'STHQJLJVM5YNDEIIOQRL0YQJIOAPR0BBXISCEEVLBZTUNVUH'
VERSION = '20200428'

In [10]:
explore = df_toronto.sample(n=1)
explore

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.38316


In [11]:
print('Latitude and longitude values of {} are {}, {}.'.format(explore.Borough[0], explore.Latitude[0],explore.Longitude[0]))

Latitude and longitude values of Central Toronto are 43.6895743, -79.38315990000001.


In [12]:
# @hidden_cell
LIMIT = 100
lat = explore.Latitude[0]
lng = explore.Longitude[0]
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    500, 
    LIMIT)

In [13]:
import requests
import json

results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Loring-Wyle Parkette,Park,43.690270,-79.383438
1,Reds,Restaurant,43.692816,-79.384504
2,Moore Park Tennis Club,Tennis Court,43.693289,-79.382900
3,Moorevale Park,Playground,43.693610,-79.383465


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [15]:
df_toronto.groupby('Borough').count()

,Neighborhood,Latitude,Longitude
Borough,,,
Central Toronto,9,9,9
Downtown Toronto,19,19,19
East Toronto,5,5,5
West Toronto,6,6,6


In [16]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 4 uniques categories.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# type your answer here

df_toronto_venues = getNearbyVenues(names=df_toronto['Borough'],latitudes=df_toronto['Latitude'], longitudes=df_toronto['Longitude'])

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [19]:
print(df_toronto_venues.shape)
df_toronto_venues.head()

(1636, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [20]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


In [21]:
# one hot encoding
onehot_toronto = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_toronto['Borough'] = df_toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [onehot_toronto.columns[-1]] + list(onehot_toronto.columns[:-1])
onehot_toronto = onehot_toronto[fixed_columns]

onehot_toronto.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
onehot_toronto.shape

(1636, 235)

In [23]:
toronto_grouped = onehot_toronto.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,...,0.016949,0.008475,0.000000,0.008475,0.000000,0.008475,0.000000,0.000000,0.000000,0.008475
1,Downtown Toronto,0.000812,0.000812,0.000812,0.001625,0.002437,0.001625,0.015435,0.001625,0.004062,...,0.000000,0.000812,0.002437,0.009748,0.001625,0.003249,0.004874,0.000000,0.000812,0.005686
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590,0.000000,0.000000,...,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.008197,0.000000,0.000000,0.016393
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006061,0.000000,...,0.000000,0.000000,0.000000,0.018182,0.000000,0.012121,0.006061,0.006061,0.000000,0.012121


In [24]:
toronto_grouped.shape

(4, 235)

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Park  0.06
2  Sandwich Place  0.06
3            Café  0.05
4     Pizza Place  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.06
2          Restaurant  0.04
3               Hotel  0.03
4  Italian Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.05
2  Italian Restaurant  0.04
3             Brewery  0.04
4                Café  0.04


----West Toronto----
                venue  freq
0                Café  0.07
1                 Bar  0.05
2         Coffee Shop  0.04
3          Restaurant  0.04
4  Italian Restaurant  0.04




In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
3,West Toronto,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park


In [28]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = len(borough_venues_sorted.index)

grouped_clustering_toronto = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering_toronto)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 3])

In [29]:
borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
3,West Toronto,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park


In [30]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,,,,,
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore


In [31]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,
M4N,Lawrence Park,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M5N,Roselawn,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M4P,Davisville North,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M5P,Forest Hill North & West,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M4R,North Toronto West,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M5R,The Annex / North Midtown / Yorkville,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M4S,Davisville,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M4T,Moore Park / Summerhill East,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym
M4V,Summerhill West / Rathnelly / South Hill / For...,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Clothing Store,Gym


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,
M4E,The Beaches,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
M4K,The Danforth West / Riverdale,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
M4L,India Bazaar / The Beaches West,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
M4M,Studio District,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore
M7Y,Business reply mail Processing CentrE,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Park,American Restaurant,Bakery,Bookstore


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,
M5A,Regent Park / Harbourfront,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M7A,Queen's Park / Ontario Provincial Government,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5B,"Garden District, Ryerson",Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5C,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5E,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5G,Central Bay Street,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M6G,Christie,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5H,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant
M5J,Harbourfront East / Union Station / Toronto Is...,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Park,American Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,
M6H,Dufferin / Dovercourt Village,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park
M6J,Little Portugal / Trinity,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park
M6K,Brockton / Parkdale Village / Exhibition Place,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park
M6P,High Park / The Junction South,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park
M6R,Parkdale / Roncesvalles,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park
M6S,Runnymede / Swansea,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Grocery Store,Bakery,Vegetarian / Vegan Restaurant,Park


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal code,,,,,,,,,,,
